In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'handwriting-recognition:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F818027%2F1400106%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240325%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240325T100207Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D433c56aba2890f226f66f7513805f66b4dde6eb0b3d3f7288e6cd8e76e3ebebeb350eb940fb4b7aa306a6990df1a9ac7b9092fd53adaa44460becc1a19b559fc1bcbd10ac4f2b372366b3cc0a859b0eb5e555ae5edf25316a8144bc25d0685dfd01d9ddfa88b066ed41d081591c2219751d9a3d16d7ddacf57d3bf3490f1e8ce428f82efdc27e58a1df214a5a1cccb2580ce6ee88a119c0b820a31fe59f1128b3dc21c11d55607da24ce82ca5b1525dc18ea12c7924a8fc31dd2e0f9441ef2c3d8d5f7b3714e47278f55dff96f8fc5dd277025848a0aecf7f09bd7f405029dbe032444000272a8505004b6277328deadccbd88fd28d1bd6f6829d1418b4f2b1f'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

#!umount /kaggle/input/ 2> /dev/null
#shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)

    # Check if the dataset already exists
    if os.path.exists(destination_path):
        print(f'Dataset already exists at {destination_path}, skipping download.')
        continue

    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


Dataset already exists at /kaggle/input/handwriting-recognition, skipping download.
Data source import complete.


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
import os

train_dir = "/kaggle/input/handwriting-recognition/train_v2/train/"

train = pd.read_csv('/kaggle/input/handwriting-recognition/written_name_train_v2.csv')
test = pd.read_csv('/kaggle/input/handwriting-recognition/written_name_test_v2.csv')

COUNT = 30000

images = []
labels = []

train_images = []
train_labels = []
train_input_length = []
train_label_length = []
train_original_text = []

valid_images = []
valid_labels = []
valid_input_length = []
valid_label_length = []
valid_original_text = []

inputs_length = []
labels_length = []

train.head()

,FILENAME,IDENTITY
0,TRAIN_00001.jpg,BALTHAZAR
1,TRAIN_00002.jpg,SIMON
2,TRAIN_00003.jpg,BENES
3,TRAIN_00004.jpg,LA LOVE
4,TRAIN_00005.jpg,DAPHNE


In [ ]:
train.describe()

,FILENAME,IDENTITY
count,330961,330396
unique,330961,100539
top,TRAIN_00001.jpg,THOMAS
freq,1,1825


In [ ]:
train.shape

(330961, 2)

In [ ]:
def preprocess_image(img):
    w, h = img.shape

    new_w = 32
    new_h = int(h * (new_w / w))
    img = cv2.resize(img, (new_h, new_w))
    w, h = img.shape

    img = img.astype('float32')

    # Converts each to (32, 128, 1)
    if w < 32:
        add_zeros = np.full((32-w, h), 255)
        img = np.concatenate((img, add_zeros))
        w, h = img.shape

    if h < 128:
        add_zeros = np.full((w, 128-h), 255)
        img = np.concatenate((img, add_zeros), axis=1)
        w, h = img.shape

    if h > 128 or w > 32:
        dim = (128,32)
        img = cv2.resize(img, dim)

    img = cv2.subtract(255, img)

    img = np.expand_dims(img, axis=2)

    # Normalize
    img = img / 255

    return img

In [ ]:
def encode_to_labels(txt):
    # encoding each output word into digits
    dig_lst = []
    for index, chara in enumerate(txt):
        dig_lst.append(char_list.index(chara))

    return dig_lst

In [ ]:
train_labels_dict = dict(zip(train.FILENAME, train.IDENTITY))

counter = 0

for each_img in os.listdir(train_dir):
    if each_img.lower().endswith('.jpg') or each_img.lower().endswith('.png'):
        # Read and resize the image
        read_img = cv2.imread(os.path.join(train_dir, each_img), cv2.IMREAD_GRAYSCALE)
        processed_img = preprocess_image(read_img)

        # Append the image and its corresponding label to the lists
        train_images.append(processed_img)
        train_labels.append(encode_to_labels(str(train_labels_dict[each_img])))

        counter += 1
        if counter >= COUNT:
          break

In [ ]:
char_list = "!\"#&'()*+,-./0123456789:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz"
print(char_list, len(char_list))

!"#&'()*+,-./0123456789:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz 78


In [ ]:
import cv2
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# Load the labels
labels_dict = dict(zip(train.FILENAME, train.IDENTITY))

train_dir = "/kaggle/input/handwriting-recognition/train_v2/train/"
train_imgs = []
labels = []


def preprocess_image(img):
    w, h = img.shape

    new_w = 32
    new_h = int(h * (new_w / w))
    img = cv2.resize(img, (new_h, new_w))
    w, h = img.shape

    img = img.astype('float32')

    # Converts each to (32, 128, 1)
    if w < 32:
        add_zeros = np.full((32-w, h), 255)
        img = np.concatenate((img, add_zeros))
        w, h = img.shape

    if h < 128:
        add_zeros = np.full((w, 128-h), 255)
        img = np.concatenate((img, add_zeros), axis=1)
        w, h = img.shape

    if h > 128 or w > 32:
        dim = (128,32)
        img = cv2.resize(img, dim)

    img = cv2.subtract(255, img)

    #img = np.expand_dims(img, axis=2)

    # Normalize
    img = img / 255

    return img


counter = 0

for each_img in os.listdir(train_dir):
    if each_img.lower().endswith('.jpg') or each_img.lower().endswith('.png'):
        # Read and resize the image
        read_img = cv2.imread(os.path.join(train_dir, each_img), cv2.IMREAD_GRAYSCALE)
        processed_img = preprocess_image(read_img)

        # Append the image and its corresponding label to the lists
        train_imgs.append(processed_img)
        labels.append(labels_dict[each_img])

        counter += 1
        if counter >= COUNT:
          break

train_imgs = [img.flatten() for img in train_imgs]
# Convert the list of images to a numpy array and normalize pixel values
train_imgs = np.array(train_imgs) / 255.0

# Convert the list of labels to a numpy array
labels = np.array(labels)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(train_imgs, labels, test_size=0.2, random_state=42)

# Flatten the images
#num_images, height, width = X_train.shape
#X_train = X_train.reshape((num_images, height * width))

#num_images, height, width = X_test.shape
#X_test = X_test.reshape((num_images, height * width))

In [ ]:
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import accuracy_score

from sklearn.metrics import confusion_matrix

model = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=0)

model.fit(X_train, y_train)

RandomForestClassifier(max_depth=10, random_state=0)

In [ ]:
y_pred = model.predict(X_test)
rf_acc = accuracy_score(y_test, y_pred)

print("Model accuracy: {0:.3f}%".format(rf_acc * 100))

Model accuracy: 3.767%


In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report
import scipy

clf = MLPClassifier(solver='lbfgs',hidden_layer_sizes=(1000,),  random_state=1 )
best_model = clf.fit(X_train, y_train)

#train['label_predicted'] =  best_model.predict(X_train)
#train['valid_predicted'] = train.apply(lambda row: row['label_predicted']==row['label'], axis=1)

print('Training Accuracy: {:3.2f} %'.format(best_model.score(X_train,y_train)*100))
y_test_pred = best_model.predict(X_train)